In [ ]:
# todo : change db from stage to prodction

In [ ]:
import os
from dotenv import load_dotenv
import xml.etree.cElementTree as ET

load_dotenv()

MYSQL_SERVER = os.getenv("MYSQL_SERVER")
MYSQL_USER_ID = os.getenv("MYSQL_USER_ID")
MYSQL_USER_PWD = os.getenv("MYSQL_USER_PWD")
MYSQL_DB = os.getenv("MYSQL_DB")


In [ ]:
# Connect to MySQL
class UseMySQL:
    # Importing MySQLdb and as a smaller name for reusing it.
    # To install MySQL Python connection use below command:
    # pip install mysqlclient
    
    import MySQLdb as _mysql
    
    def __init__(self, host, user, password, db):
        """
        Initializing MySQL details in init method.
        Args:
            self
            host: Host name of the machine.
            user: Username of mySQL DB.
            password: Password of mySQL DB.
            db: Database name of mySQL DB.
        """
        self._host = host
        self._user = user
        self._password = password
        self._db = db

    def __enter__(self):
        """
        This data model/dunder method is used
        to employ auto open of my sql connection
        and cursor inside 'with' and in turn
        close them inside the same with the help
        of exit method in order to avoid not so
        closed database connections.
        Args:
            self
        Returns:
            self.cursor
        """
        self._conn = self._mysql.connect(
            host=self._host, user=self._user, passwd=self._password, db=self._db)
        self.cursor = self._conn.cursor()
        return self.cursor

    def __exit__(self, exception_type, exception_value, traceback):
        """
        This data model/dunder method is used to employ
        auto close of connection and cursor as with block ends.
        Args:
            self
            exception_type: Type of Exception.
            exception_value: Exception Value.
            traceback: Traceback value.
        """
        self.cursor.close()
        self._conn.close()


In [ ]:
%%time
# run query
with UseMySQL(host=MYSQL_SERVER, user=MYSQL_USER_ID, password=MYSQL_USER_PWD, db=MYSQL_DB) as mySqlObj:
    cursor = mySqlObj
    cursor.execute("""
                    select 
                    rx3_documents.document_srl, 
                    rx3_documents.module_srl, 
                    rx3_modules.module, 
                    left(rx3_documents.regdate, 8),
                    rx3_modules.mid,
                    cast(concat('https://www.<YOUR-DOMAIN>.com/', mid, '/', document_srl) as CHAR(10000) CHARACTER SET utf8) as loc,
                    datediff(curdate(), rx3_documents.regdate) as dd,
                    CASE
                        WHEN datediff(curdate(), rx3_documents.regdate) <= 7 THEN 'daily'
                        WHEN datediff(curdate(), rx3_documents.regdate) > 7 and datediff(curdate(), rx3_documents.regdate) <= 30 THEN 'weekly'
                        WHEN datediff(curdate(), rx3_documents.regdate) > 30 and datediff(curdate(), rx3_documents.regdate) <= 365 THEN 'monthly'
                        WHEN datediff(curdate(), rx3_documents.regdate) > 365 THEN 'yearly'
                    END as changefreq,
                    CASE
                        WHEN datediff(curdate(), rx3_documents.regdate) <= 7 THEN 0.8
                        WHEN datediff(curdate(), rx3_documents.regdate) > 7 and datediff(curdate(), rx3_documents.regdate) <= 30 THEN 0.7
                        WHEN datediff(curdate(), rx3_documents.regdate) > 30 and datediff(curdate(), rx3_documents.regdate) <= 365 THEN 0.6
                        WHEN datediff(curdate(), rx3_documents.regdate) > 365 THEN 0.5
                    END as priority
                    from rx3_documents
                        inner join rx3_modules 
                            on rx3_documents.module_srl = rx3_modules.module_srl
                    where 
                    rx3_modules.module like 'page'
                    or mid in ('board_FreeTalk', 'board_Photo', 'board_Local')
                    order by rx3_documents.regdate desc
                    limit 49500; """)
    res = cursor.fetchall()  # tuple type

In [ ]:
for tup1 in res[0:5]:  # check 5 result
    s = tup1[3]
    s = s[:4] + '-' + s[4:]
    s = s[:7] + '-' + s[7:]
    print(tup1[5], s, tup1[7], tup1[8]) # fetch field values

In [ ]:
print(len(res))

# Final code - generate xpressegnine sitemap

In [ ]:
def set_date(raw_date):
    raw_date = raw_date[:4] + '-' + raw_date[4:]
    val_date = raw_date[:7] + '-' + raw_date[7:]
    return val_date

In [ ]:
%%time
import xml.etree.cElementTree as ET

d = {'xmlns':'http://www.sitemaps.org/schemas/sitemap/0.9'}  # root dict
root = ET.Element("urlset", d)

for tup1 in res:
    url = ET.SubElement(root, "url")
    loc = tup1[5]
    lastmod = set_date(tup1[3]) 
    changefreq = tup1[7]
    priority = str(tup1[8])
    ET.SubElement(url, "loc").text = loc
    ET.SubElement(url, "lastmod").text = lastmod
    ET.SubElement(url, "changefreq").text = changefreq
    ET.SubElement(url, "priority").text = priority

tree = ET.ElementTree(root)

# set ET to str
str_xml = ET.tostring(root, encoding="utf-8", xml_declaration=True).decode('utf-8')

# write to siteml.xml file
text_file = open("sitemap.xml", "w", encoding="utf-8")
text_file.write(str_xml)
text_file.close()
